In [1]:
import io
import arcpy
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.widgets import MapView
from zipfile import ZipFile
import os
from osgeo import ogr
from arcgis.geometry import SpatialReference
import sys

from arcgis import GIS
gis = GIS()

In [2]:
#ndawn monthly normals for all stations
ndawn_link = r'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=224&station=159&station=10&station=229&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=218&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=210&station=201&station=137&station=124&station=143&station=17&station=85&station=226&station=140&station=134&station=18&station=136&station=219&station=65&station=104&station=99&station=192&station=19&station=227&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=211&station=172&station=139&station=158&station=23&station=157&station=220&station=62&station=86&station=24&station=89&station=126&station=223&station=167&station=93&station=183&station=90&station=25&station=205&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=217&station=80&station=33&station=59&station=153&station=105&station=82&station=225&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=209&station=141&station=109&station=36&station=207&station=79&station=193&station=71&station=212&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=228&station=69&station=194&station=145&station=214&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=216&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=206&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=213&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=221&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=222&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=208&station=57&station=149&station=148&station=202&station=215&station=110&variable=ddavt&year=2024&ttype=daily&quick_pick=30_d&begin_date=2024-10-01&end_date=2024-10-31'

#call to the api
ndawn_response = requests.get(ndawn_link, stream=True).content

In [9]:
#save as dataframe, and cleaning csv a bit to make more readable
df1 = pd.read_csv(io.StringIO(ndawn_response.decode('utf-8')), skiprows=[0, 1, 2, 4], index_col=False)
df1

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Avg Temp Flag
0,Ada,47.32119,-96.51406,910,2024,10,16,50.790,NaN
1,Ada,47.32119,-96.51406,910,2024,10,17,60.620,NaN
2,Ada,47.32119,-96.51406,910,2024,10,18,52.815,NaN
3,Ada,47.32119,-96.51406,910,2024,10,19,47.294,NaN
4,Ada,47.32119,-96.51406,910,2024,10,20,60.150,NaN
...,...,...,...,...,...,...,...,...,...
6507,Zeeland,46.01351,-99.68768,2070,2024,11,10,43.395,NaN
6508,Zeeland,46.01351,-99.68768,2070,2024,11,11,32.352,NaN
6509,Zeeland,46.01351,-99.68768,2070,2024,11,12,41.021,NaN
6510,Zeeland,46.01351,-99.68768,2070,2024,11,13,34.612,NaN


In [27]:
#our data is ready, but it isn't spatial yet so converting it to a spatially enabled df now-->
sedf = pd.DataFrame.spatial.from_xy(df=df1, x_column="Longitude", y_column="Latitude", sr=4326)

sedf.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Avg Temp Flag,SHAPE
0,Ada,47.32119,-96.51406,910,2024,10,16,50.790,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
1,Ada,47.32119,-96.51406,910,2024,10,17,60.620,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
2,Ada,47.32119,-96.51406,910,2024,10,18,52.815,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
3,Ada,47.32119,-96.51406,910,2024,10,19,47.294,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
4,Ada,47.32119,-96.51406,910,2024,10,20,60.150,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."


In [36]:
#Dropping two stations that did not have 30 days worth of data
indexName = sedf[(sedf['Station Name'] == 'Beardsley') & (sedf['Station Name'] == 'Morris')].index

sedf.drop(indexName, inplace = True)
sedf.head(90)

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Avg Temp Flag,SHAPE
0,Ada,47.32119,-96.51406,910,2024,10,16,50.790,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
1,Ada,47.32119,-96.51406,910,2024,10,17,60.620,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
2,Ada,47.32119,-96.51406,910,2024,10,18,52.815,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
3,Ada,47.32119,-96.51406,910,2024,10,19,47.294,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
4,Ada,47.32119,-96.51406,910,2024,10,20,60.150,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
...,...,...,...,...,...,...,...,...,...,...
85,Alamo,48.54652,-103.47186,2157,2024,11,10,36.318,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
86,Alamo,48.54652,-103.47186,2157,2024,11,11,33.331,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
87,Alamo,48.54652,-103.47186,2157,2024,11,12,33.962,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
88,Alamo,48.54652,-103.47186,2157,2024,11,13,34.570,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."


In [29]:
#Grouping the stations so that we have just one row per station, rather than 30
updated_sedf = sedf.groupby('Station Name').agg({'Latitude': 'mean', 'Longitude': 'mean', 'Elevation': 'max', 'Year': 'max', 'Month': 'max', 'Day': 'count', 'Avg Temp': 'mean'})

,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp
Station Name,,,,,,,
Ada,47.32119,-96.51406,910,2024,11,30,44.942967
Adams,48.49988,-98.07588,1580,2024,11,30,41.285033
Alamo,48.54652,-103.47186,2157,2024,11,30,40.888000
Alexander,47.75056,-103.73358,2202,2024,11,30,42.882167
Alvarado,48.24594,-97.02153,809,2024,11,30,43.458167
...,...,...,...,...,...,...,...
Wolford,48.51603,-99.62441,1610,2024,11,30,40.950000
Wolverton,46.56545,-96.68726,937,2024,11,30,44.836133
Ypsilanti,46.77638,-98.52323,1484,2024,11,30,44.289667


In [32]:
#make a basic map
basic_map = gis.map()

updated_sedf.spatial.plot(
    map_widget=basic_map,
    renderer_type="s",
    symbol_type="simple",
    symbol_style="d",
    marker_size=10,
)

#showing it below
basic_map


MapView(layout=Layout(height='400px', width='100%'))

In [33]:
#go from sedf to feature class here with a given directory-- ndawnfc shapefile will be created and placed in directory
sedf.spatial.to_featureclass(r'C:\Users\bende287\Documents\GIS5571\ndawnfcmonthly')

'C:\\Users\\bende287\\Documents\\GIS5571\\ndawnfcmonthly.shp'

In [34]:
#Interpolation Methods:
#IDW 
arcpy.ga.IDW(
    in_features="ndawnfcmonthly",
    z_field="avg_temp",
    out_ga_layer="IDWndawn2",
    out_raster=None,
    cell_size=0.021010908,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=3.42563109432527 S_MINOR=3.42563109432527 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'IDWndawn2'>

In [35]:
#for mean temp interpolation-- Kriging-exponential and Kriging-spherical have the best precision
#ordinary Kriging module, using the spherical semi-variogram model
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="ndawnfcmonthly",
        z_field="avg_temp",
        kriging_model="Spherical # # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb\Kriging_sphere")

In [ ]:
#ordinary Kriging module, using the exponential semi-variogram model
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="ndawnfcmonthly",
        z_field="avg_temp",
        kriging_model="Exponential 0.021011 # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb\Kriging_expo")

##The following code will include min and max temperature since I think I originally read the lab instructions wrong.. just covering my bases.

In [3]:

#ndawn monthly normals for all stations
ndawn_link2 = r'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=224&station=159&station=10&station=229&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=218&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=210&station=201&station=137&station=124&station=143&station=17&station=85&station=226&station=140&station=134&station=18&station=136&station=219&station=65&station=104&station=99&station=192&station=19&station=227&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=211&station=172&station=139&station=158&station=23&station=157&station=220&station=62&station=86&station=24&station=89&station=126&station=223&station=167&station=93&station=183&station=90&station=25&station=205&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=217&station=80&station=33&station=59&station=153&station=105&station=82&station=225&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=209&station=141&station=109&station=36&station=207&station=79&station=193&station=71&station=212&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=228&station=69&station=194&station=145&station=214&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=216&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=206&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=213&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=221&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=222&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=208&station=57&station=149&station=148&station=202&station=215&station=110&variable=ddmxt&variable=ddmnt&variable=ddavt&year=2024&ttype=daily&quick_pick=30_d&begin_date=2024-11-25&end_date=2024-11-25'

#call to the api
ndawn_response2 = requests.get(ndawn_link2, stream=True).content

In [5]:
#save as dataframe, and cleaning csv a bit to make more readable
df2 = pd.read_csv(io.StringIO(ndawn_response2.decode('utf-8')), skiprows=[0, 1, 2, 4], index_col=False)
df2

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Max Temp Flag,Min Temp,Min Temp Flag,Avg Temp,Avg Temp Flag
0,Ada,47.32119,-96.51406,910,2024,10,27,60.224,NaN,32.099,NaN,46.162,NaN
1,Ada,47.32119,-96.51406,910,2024,10,28,65.480,NaN,44.130,NaN,54.805,NaN
2,Ada,47.32119,-96.51406,910,2024,10,29,58.658,NaN,43.349,NaN,51.004,NaN
3,Ada,47.32119,-96.51406,910,2024,10,30,46.328,NaN,28.666,NaN,37.497,NaN
4,Ada,47.32119,-96.51406,910,2024,10,31,41.322,NaN,23.778,NaN,32.550,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6529,Zeeland,46.01351,-99.68768,2070,2024,11,21,23.389,NaN,7.286,NaN,15.338,NaN
6530,Zeeland,46.01351,-99.68768,2070,2024,11,22,37.450,NaN,7.988,NaN,22.719,NaN
6531,Zeeland,46.01351,-99.68768,2070,2024,11,23,27.529,NaN,21.335,NaN,24.432,NaN
6532,Zeeland,46.01351,-99.68768,2070,2024,11,24,26.530,NaN,11.894,NaN,19.212,NaN


In [7]:
#our data is ready, but it isn't spatial yet so converting it to a spatially enabled df now-->
sedf2 = pd.DataFrame.spatial.from_xy(df=df2, x_column="Longitude", y_column="Latitude", sr=4326)

sedf2.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Max Temp Flag,Min Temp,Min Temp Flag,Avg Temp,Avg Temp Flag,SHAPE
0,Ada,47.32119,-96.51406,910,2024,10,27,60.224,NaN,32.099,NaN,46.162,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
1,Ada,47.32119,-96.51406,910,2024,10,28,65.480,NaN,44.130,NaN,54.805,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
2,Ada,47.32119,-96.51406,910,2024,10,29,58.658,NaN,43.349,NaN,51.004,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
3,Ada,47.32119,-96.51406,910,2024,10,30,46.328,NaN,28.666,NaN,37.497,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
4,Ada,47.32119,-96.51406,910,2024,10,31,41.322,NaN,23.778,NaN,32.550,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."


In [8]:
#Dropping two stations that did not have 30 days worth of data
indexName2 = sedf2[(sedf2['Station Name'] == 'Beardsley') & (sedf2['Station Name'] == 'Morris')].index

sedf2.drop(indexName2, inplace = True)
sedf2.head(90)

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Max Temp Flag,Min Temp,Min Temp Flag,Avg Temp,Avg Temp Flag,SHAPE
0,Ada,47.32119,-96.51406,910,2024,10,27,60.224,NaN,32.099,NaN,46.162,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
1,Ada,47.32119,-96.51406,910,2024,10,28,65.480,NaN,44.130,NaN,54.805,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
2,Ada,47.32119,-96.51406,910,2024,10,29,58.658,NaN,43.349,NaN,51.004,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
3,Ada,47.32119,-96.51406,910,2024,10,30,46.328,NaN,28.666,NaN,37.497,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
4,Ada,47.32119,-96.51406,910,2024,10,31,41.322,NaN,23.778,NaN,32.550,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Alamo,48.54652,-103.47186,2157,2024,11,21,23.295,NaN,0.140,NaN,11.718,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
86,Alamo,48.54652,-103.47186,2157,2024,11,22,26.188,NaN,14.684,NaN,20.436,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
87,Alamo,48.54652,-103.47186,2157,2024,11,23,20.813,NaN,14.792,NaN,17.803,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."
88,Alamo,48.54652,-103.47186,2157,2024,11,24,24.208,NaN,3.794,NaN,14.001,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -103..."


In [10]:
#Grouping the stations so that we have just one row per station, rather than 30
updated_sedf2 = sedf2.groupby('Station Name').agg({'Latitude': 'mean', 'Longitude': 'mean', 'Elevation': 'max', 'Year': 'max', 'Month': 'max', 'Day': 'count', 'Avg Temp': 'mean', 'Min Temp': 'mean', 'Max Temp': 'mean'})

In [15]:
#go from sedf to feature class here with a given directory-- ndawnfc shapefile will be created and placed in directory
sedf2.spatial.to_featureclass(r'C:\Users\bende287\Documents\GIS5571\ndawnfcminmax')

'C:\\Users\\bende287\\Documents\\GIS5571\\ndawnfcminmax.shp'

In [16]:
#Interpolation Methods:
#IDW 
arcpy.ga.IDW(
    in_features="ndawnfcminmax",
    z_field="max_temp",
    out_ga_layer="IDWndawnmax",
    out_raster=None,
    cell_size=0.021010908,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=3.42563109432527 S_MINOR=3.42563109432527 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'IDWndawnmax'>

In [19]:
#for temp interpolation-- Kriging-exponential and Kriging-spherical have the best precision
#ordinary Kriging module, using the spherical semi-variogram model
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="ndawnfcminmax",
        z_field="max_temp",
        kriging_model="Spherical # # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb\Kriging_sphereMAX")

In [20]:
#ordinary Kriging module, using the exponential semi-variogram model
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="ndawnfcminmax",
        z_field="max_temp",
        kriging_model="Exponential 0.021011 # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb\Kriging_expoMAX")

In [7]:
#Interpolation Methods:
#IDW 
arcpy.ga.IDW(
    in_features="ndawnfcminmax",
    z_field="max_temp",
    out_ga_layer="IDWmax",
    out_raster=None,
    cell_size=0.021010908,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=3.42563109432527 S_MINOR=3.42563109432527 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'IDWmax'>

In [8]:
#Interpolation Methods:
#IDW 
arcpy.ga.IDW(
    in_features="ndawnfcminmax",
    z_field="min_temp",
    out_ga_layer="IDWmin",
    out_raster=None,
    cell_size=0.021010908,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=3.42563109432527 S_MINOR=3.42563109432527 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result 'IDWmin'>

In [5]:
#ordinary Kriging module, using the exponential semi-variogram model
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="ndawnfcminmax",
        z_field="min_temp",
        kriging_model="Exponential 0.021011 # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb\Kriging_expoMIN")

In [6]:
#for temp interpolation-- Kriging-exponential and Kriging-spherical have the best precision
#ordinary Kriging module, using the spherical semi-variogram model
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="ndawnfcminmax",
        z_field="min_temp",
        kriging_model="Spherical # # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\bende287\Documents\ArcGIS\Projects\5571Lab3\5571Lab3.gdb\Kriging_sphereMIN")